# Coursera Data Science Capstone Project

Week 5 Assignment - The Battle of Neighbourhoods W2

## Data Wrangling Notebook

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import googlemaps

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from urllib.request import urlopen as ureq
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup as soup

from dotenv import load_dotenv, dotenv_values

In [2]:
load_dotenv()
config = dotenv_values(".env")

## General Stats Data
<a id='generalstatsdata'></a> 

In [3]:
def get_content_from_url(url):
    """
    Function to load the DOM contents for scrapping

        Parameters:
            url (string): URL string for the site to scrap
        
        Returns:
            html (class bytes): DOM object
    """
    
    try:
        u_client = ureq(url)
        html = u_client.read()
        u_client.close()
    except HTTPError as e:
        print(f'There has been a problem connecting to the web site. ERROR: {e.code}')
    except URLError as e:
        print(f'There has been a problem reaching the web site. ERROR: {e.code}')
    else:
        print(f'Content loaded!')
        return html

In [4]:
def get_rows_cols(src):
    """
    Function to return a list of names for use to create a DataFrame

        Parameters:
            src (bs4.element.Tag): BeautifulSoup element to extract the tag names 

        Returns:
            trg_list (list): A list containing the names
    """
    trg_list = []
    for el in src.find_all('th'):
        trg_list.append(el.text.strip())
    return trg_list

In [5]:
def fill_df(src_data, cols_list, rows_list):
    """
    Function to fill in a DataFrame
        
        Parameters:
            src_data (bs4.element.Tag): BeautifulSoup data source tag/element
            cols_list (list): A list containing the column names
            rows_list (list): A list containing the row name
        
        Returns:
            df (DataFrame): A Pandas DataFrame
    """
    df = pd.DataFrame(columns=cols_list)
    
    for data in src_data.tbody.find_all('tr'):
        temp_dict = {}
        for i, td in enumerate(cols_list):
            temp_dict[td] = data.find_all('td')[i].string
            
        df = df.append(temp_dict, ignore_index=True)
    
    df.index=rows_list
    return df

In [6]:
dorset_insights_url = 'https://apps.geowessex.com/insights/AreaProfiles/PreUnitaryAuthority/bournemouth'

dorset_insights_html = get_content_from_url(dorset_insights_url)

Content loaded!


In [7]:
dorset_insights_soup = soup(dorset_insights_html, 'html.parser')
# print(dorset_insights_soup)

In [8]:
# Get the features and indices names for the Population and Business_Units Dataframes

population_soup_table = dorset_insights_soup.find_all('table')[0]
business_units_soup_table = dorset_insights_soup.find_all('table')[10]

population_table_rows = population_soup_table.find_all('tbody')[0]
population_table_cols = population_soup_table.find_all('thead')[0].find_all('tr')[1]
population_cols = get_rows_cols(population_table_cols)
population_rows = get_rows_cols(population_table_rows)

business_units_table_rows = business_units_soup_table.find_all('tbody')[0]
business_units_table_cols = business_units_soup_table.find_all('thead')[0]
business_units_cols = get_rows_cols(business_units_table_cols)
business_units_rows = get_rows_cols(business_units_table_rows)

In [9]:
# Fill in Population Dataframe

df_population = fill_df(population_soup_table, population_cols, population_rows)
df_population

,Total,Percentage,Dorset Council,England and Wales
All Persons,"194,360",-,"376,480","59,115,810"
All Males,"98,100",50.5%,48.9%,49.4%
All Females,"96,250",49.5%,51.1%,50.6%
0-15 yrs,"32,970",17.0%,15.9%,19.1%
16-64 yrs,"126,160",64.9%,55.4%,62.5%
65+ yrs,"35,230",18.1%,28.6%,18.3%
Male 0-15 yrs,"16,830",8.7%,8.1%,9.8%
Male 16-64 yrs,"65,170",33.5%,27.5%,31.2%
Male 65+ yrs,"16,110",8.3%,13.2%,8.4%
Female 0-15 yrs,"16,140",8.3%,7.8%,9.3%


**Save to file**

In [10]:
df_population.to_csv('general_population_stats.csv', index=True)

In [11]:
# Fill in Business_Units Dataframe

df_business_units = fill_df(business_units_soup_table, business_units_cols, business_units_rows)
df_business_units

,Total,Percentage
Total businesses,"8,080",-
"Wholesale, retail and repair","1,420",17.57%
Construction,"1,070",13.24%
"Professional, scientific and technical activities","1,045",12.93%
Accommodation and food services,730,9.03%
Administrative and support service activities,700,8.66%
Information and communications,630,7.80%
Human heath and social work activites,570,7.05%
Real estate activities,350,4.33%
Manufacturing,280,3.47%


**Save to file**

In [12]:
df_business_units.to_csv('general_business_stats.csv', index=True)

## Rightmove Data
<a id='rightmovedata'></a> 

In [13]:
rightmove_url_page_1 = 'https://www.rightmove.co.uk/commercial-property-to-let/find.html?locationIdentifier=REGION%5E194&propertyTypes=serviced-office%2Coffice&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&areaSizeUnit=sqft&priceType=pcm&keywords='

rightmove_url_page_2 = 'https://www.rightmove.co.uk/commercial-property-to-let/find.html?locationIdentifier=REGION%5E194&index=24&propertyTypes=serviced-office%2Coffice&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&areaSizeUnit=sqft&priceType=pcm&keywords='

rightmove_url_page_3 = 'https://www.rightmove.co.uk/commercial-property-to-let/find.html?locationIdentifier=REGION%5E194&index=48&propertyTypes=serviced-office%2Coffice&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&areaSizeUnit=sqft&priceType=pcm&keywords='

rightmove_url_page_4 = 'https://www.rightmove.co.uk/commercial-property-to-let/find.html?locationIdentifier=REGION%5E194&index=72&propertyTypes=serviced-office%2Coffice&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&areaSizeUnit=sqft&priceType=pcm&keywords='

rightmove_html_p1 = get_content_from_url(rightmove_url_page_1)
rightmove_html_p2 = get_content_from_url(rightmove_url_page_2)
rightmove_html_p3 = get_content_from_url(rightmove_url_page_3)
rightmove_html_p4 = get_content_from_url(rightmove_url_page_4)

rightmove_pages = (rightmove_html_p1, rightmove_html_p2, rightmove_html_p3, rightmove_html_p4)

Content loaded!
Content loaded!
Content loaded!
Content loaded!


In [14]:
property_cards = []
for page in rightmove_pages:
    html = soup(page, 'html.parser')
    property_card = html.find_all(class_='propertyCard')
    property_cards.extend(property_card)

In [15]:
df_rightmove = pd.DataFrame(columns=['id', 'address', 'rent', 'size'])

for property in property_cards:
    id = property.find('a', class_='propertyCard-anchor').attrs
    address = property.find('address', class_='propertyCard-address').contents[-2].text
    rent = property.find(class_='propertyCard-priceValue').text
    size = property.find(class_='propertyCard-commercial-sizing--link').text
    
    if address != '':
        d = {
            'id': id['id'][4:],
            'address': address,
            'rent': rent,
            'size': size
        }
        df_rightmove = df_rightmove.append(d, ignore_index=True)

df_rightmove

,id,address,rent,size
0,88973570,"Roddis House, 4-10 Old Christchurch Road, Bour...",£942 pcm,904 sq. ft.
1,83154023,"Fir Vale Road, Bournemouth, Dorset, BH1",POA,"491–35,200 sq. ft."
2,85052380,"The Helm, Holdenhurst Road, Bournemouth, Dorse...",POA,"8,417–34,488 sq. ft."
3,85271110,"Richmond House, Richmond Hill, Bournemouth, BH...",POA,"1,336–12,974 sq. ft."
4,91282550,"Clarendon Business Centre Clifton House, 10, P...","£2,190 pcm","500–12,870 sq. ft."
...,...,...,...,...
93,91282442,"Platinum House, 23 , Hinton Road ,BH1 2EF",£200 pcm,50–80 sq. ft.
94,91282565,"Clarendon Business Centre Clifton House, 10, P...",£365 pcm,50–80 sq. ft.
95,95494049,"1 St Stephen's Court, St Stephen's Road, Bourn...","£3,833 pcm","2,881 sq. ft."
96,100022672,"Manor Road, Bournemouth, Dorset, BH1",£338 pcm,


Now I need to add the geolocation data, for that I'll be using Google Maps API

In [16]:
GC_API_KEY = config['GC_API_KEY']
gmaps_key = googlemaps.Client(key=GC_API_KEY)

In [17]:
address = df_rightmove['address'][0]
location = gmaps_key.geocode(address)

In [18]:
df_geocoords = pd.DataFrame(columns=['lat','lng'])

for address in df_rightmove['address']:
    try:
        location = gmaps_key.geocode(address)
        coords = {
            'lat': location[0]['geometry']['location']['lat'],
            'lng': location[0]['geometry']['location']['lng']
        }
        df_geocoords = df_geocoords.append(coords, ignore_index=True)
    except:
        coords = {
            'lat': 'None',
            'lng': 'None'
        }
        df_geocoords = df_geocoords.append(coords, ignore_index=True)
        
df_geocoords

,lat,lng
0,50.719971,-1.878147
1,50.721864,-1.874077
2,50.723514,-1.865190
3,50.721857,-1.877716
4,50.719832,-1.886341
...,...,...
93,50.719047,-1.873558
94,50.719832,-1.886341
95,50.722988,-1.882427
96,50.721178,-1.854880


In [19]:
df_offices = pd.concat([df_rightmove, df_geocoords], axis=1)
df_offices = df_offices[['id', 'address', 'lat','lng','rent','size']]
df_offices

,id,address,lat,lng,rent,size
0,88973570,"Roddis House, 4-10 Old Christchurch Road, Bour...",50.719971,-1.878147,£942 pcm,904 sq. ft.
1,83154023,"Fir Vale Road, Bournemouth, Dorset, BH1",50.721864,-1.874077,POA,"491–35,200 sq. ft."
2,85052380,"The Helm, Holdenhurst Road, Bournemouth, Dorse...",50.723514,-1.865190,POA,"8,417–34,488 sq. ft."
3,85271110,"Richmond House, Richmond Hill, Bournemouth, BH...",50.721857,-1.877716,POA,"1,336–12,974 sq. ft."
4,91282550,"Clarendon Business Centre Clifton House, 10, P...",50.719832,-1.886341,"£2,190 pcm","500–12,870 sq. ft."
...,...,...,...,...,...,...
93,91282442,"Platinum House, 23 , Hinton Road ,BH1 2EF",50.719047,-1.873558,£200 pcm,50–80 sq. ft.
94,91282565,"Clarendon Business Centre Clifton House, 10, P...",50.719832,-1.886341,£365 pcm,50–80 sq. ft.
95,95494049,"1 St Stephen's Court, St Stephen's Road, Bourn...",50.722988,-1.882427,"£3,833 pcm","2,881 sq. ft."
96,100022672,"Manor Road, Bournemouth, Dorset, BH1",50.721178,-1.854880,£338 pcm,


**Save to file**

In [20]:
df_offices.to_csv('potential_offices.csv', index=False)

## Foursquares Data
<a id='foursquaresdata'></a> 

In [21]:
CLIENT_ID = config['FS_CLIENT_ID']
CLIENT_SECRET = config['FS_CLIENT_SECRET']
ACCESS_TOKEN = config['FS_ACCESS_TOKEN']
VERSION = '20210101'

**Plot points of interest on a map**

In [22]:
radius = 100
limit = 5
index = np.random.randint(0, df_offices.shape[0])
office_location = {
    'lat': df_offices.loc[index]['lat'],
    'lng': df_offices.loc[index]['lng']
}

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, office_location['lat'], office_location['lng'], ACCESS_TOKEN, VERSION, radius, limit)

results = requests.get(url).json()
df = json_normalize(results['response']['venues'])

def plot_map(location):
    folium_map = folium.Map(location=[location['lat'], location['lng']], zoom_start=14)
    # Big marker circle I would like to disable the hand cursor
    folium.Circle([location['lat'], location['lng']], 
                    radius * 1.2,
                    fill=True,
                    fill_color='#f72585',
                    fill_opacity=0.5,
                    stroke=False).add_to(folium_map)

    if len(df.index) > 0 :
        # All the other markers on the map
        for venue, lat, lon in zip(df['name'], df['location.lat'], df['location.lng']):
            label_str = '{}'.format(venue)
            label = folium.Popup(label_str, parse_html=True)
            folium.CircleMarker(
                [lat, lon],
                radius=5,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7).add_to(folium_map)
    else:
        print('Not enough data to plot markers on the map')

    return folium_map

plot_map(office_location)

Plot office location data

In [23]:
geolocator = Nominatim(user_agent='foursquare_agent')
city = 'Bournemouth, Dorset'
search_terms = ['bus','restaurant','bar','pub','park','gym','food','pharmacy','shop']

df_venues = pd.DataFrame()
for venue in search_terms:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&oauth_token={}&v={}&near={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION, city, venue)

    results = requests.get(url).json()
    df = json_normalize(results['response']['venues'])
    df_venues = df_venues.append(df)
df_venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4c9e1438542b224b2925f09f,Bus Stop,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1611445724,False,Bournemouth Gardens,Gervis Rd,50.720050,-1.878387,"[{'label': 'display', 'lat': 50.72004991352224...",BH 1 2,GB,Bournemouth,Dorset,United Kingdom,"[Bournemouth Gardens (Gervis Rd), Bournemouth,...",NaN,NaN
1,4e282b3fd4c0e361d9db0e87,Bus Stop,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1611445724,False,Bourne Avenue,NaN,50.720999,-1.880725,"[{'label': 'display', 'lat': 50.72099925094467...",NaN,GB,Bournemouth,Dorset,United Kingdom,"[Bourne Avenue, Bournemouth, Dorset]",NaN,NaN
2,4cd812e553a7f04d8a6d6116,Bus stop - cranborne house,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1611445724,False,NaN,NaN,50.724271,-1.867263,"[{'label': 'display', 'lat': 50.72427056882155...",NaN,GB,Bournemouth,Dorset,United Kingdom,"[Bournemouth, Dorset]",NaN,NaN
3,4d7b7ccecf3f3704925d3c40,Bournemouth Sq. Bus Stop,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1611445724,False,NaN,NaN,50.719905,-1.878513,"[{'label': 'display', 'lat': 50.71990476870037...",NaN,GB,NaN,NaN,United Kingdom,NaN,NaN,NaN
4,4da59b0d0cb6d75e29fae504,Cememtery Junction Bus Stop,"[{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...",v-1611445724,False,NaN,NaN,50.731591,-1.873358,"[{'label': 'display', 'lat': 50.73159058826061...",NaN,GB,Bournemouth,Dorset,United Kingdom,"[Bournemouth, Dorset]",NaN,NaN


**Remove duplicate venues**

In [24]:
print(f'Before: {df_venues.shape}')
df_venues.drop_duplicates(subset=['id'], inplace=True)
df_venues.reset_index(inplace=True)
print(f'After: {df_venues.shape}')

Before: (270, 18)
After: (265, 19)


**Check for null values**

In [25]:
df_venues.isnull().sum(axis = 0)

index                          0
id                             0
name                           0
categories                     0
referralId                     0
hasPerk                        0
location.address             104
location.crossStreet         236
location.lat                   0
location.lng                   0
location.labeledLatLngs        0
location.postalCode          114
location.cc                    0
location.city                 79
location.state                81
location.country               0
location.formattedAddress     68
venuePage.id                 260
location.neighborhood        264
dtype: int64

**Plot venues on the map**

In [26]:
def plot_venues_on_map():
    """
        Draws a folium map and plots the venue markers in it

        Parameters: None

        Return:
            map_folium: A folium map
    """
    address = '8 Lansdowne Road, Bournemouth, UK'
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    map_folium = folium.Map(
        location=[latitude, longitude],
        zoom_start=14,
        # tiles='cartodbpositron'
    )

    if len(df.index) > 0 :
        # All the other markers on the map
        for venue, lat, lon in zip(df_venues['name'], df_venues['location.lat'], df_venues['location.lng']):
            label_str = '{}'.format(venue)
            label = folium.Popup(label_str, parse_html=True)
            folium.CircleMarker(
                [lat, lon],
                radius=5,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=0.7).add_to(map_folium)
    else:
        print('Not enough data to plot markers on the map')

    return map_folium

plot_venues_on_map()

**Clean venue categories column**

In [27]:
df_venues['categories']

0      [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...
1      [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...
2      [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...
3      [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...
4      [{'id': '52f2ab2ebcbc57f1066b8b4f', 'name': 'B...
                             ...                        
260    [{'id': '4bf58dd8d48988d101951735', 'name': 'T...
261    [{'id': '4bf58dd8d48988d102951735', 'name': 'A...
262    [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...
263    [{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...
264    [{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...
Name: categories, Length: 265, dtype: object

Some rows don't have a category so I'll drop those entries, as they won't probyde any benefefits to our project 

In [28]:
indices_to_drop = []
for i, cat in enumerate(df_venues['categories']):
    if len(cat) == 0:
        indices_to_drop.append(i)

df_venues.drop(indices_to_drop, inplace=True)
df_venues.reset_index(inplace=True)

print(f'Number of rows to drop: {len(indices_to_drop)}')
print(f'Venues DF shape: {df_venues.shape}')

Number of rows to drop: 9
Venues DF shape: (256, 20)


Now we can clean the categories column

In [29]:
categories_list = []

for cat in df_venues['categories']:
    categories_list.append(cat[0]['name'])

cat_series = pd.Series(categories_list)
df_venues['categories'] = cat_series
df_venues['categories'].to_frame()

,categories
0,Bus Stop
1,Bus Stop
2,Bus Stop
3,Bus Stop
4,Bus Stop
...,...
251,Thrift / Vintage Store
252,Accessories Store
253,Coffee Shop
254,Mobile Phone Shop


In [30]:
df_venues['categories'].value_counts(dropna=False).to_frame()

,categories
Pharmacy,29
Bus Stop,22
Grocery Store,20
Pub,16
Bar,15
...,...
Business Service,1
Climbing Gym,1
Lebanese Restaurant,1
Miscellaneous Shop,1


Remove not relevant venues to our study

In [31]:
df_venues.drop(df_venues[df_venues['categories']=='Pharmacy'].index, inplace=True)
df_venues.drop(df_venues[df_venues['categories']=='Non-Profit'].index, inplace=True)
df_venues.shape

(226, 20)

**Save to file**

In [32]:
df_venues.to_csv('bournemouth_venues.csv', index=False)